## Basic Lib to parse PSO XML and Read PSO Schedule using API
this set of functions is used mainly in the following notebooks:
<br/>01 - PSO File Schedule Analyzer
<br/>02 - PSO Plan Scheduler Analyzer

and it's meant as a basis to extract the data from a PSO schedule according to the Scheduling Schema pdf file

In [1]:
from datetime import timedelta
import isodate
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import seaborn as sns
import requests


# Function(s) to parse XML and extract <Record> records into a DataFrame using a file or plan from dataset

In [2]:
def parse_records_to_dataframe_by_file(xml_file: str, record_tag: str):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    return parse_records_to_dataframe(root, record_tag)


def parse_records_to_dataframe_by_string(xml_str: str, record_tag: str):
    root = ET.fromstring(xml_str)
    return parse_records_to_dataframe(root, record_tag)

# Function to parse XML and extract <record_tag> records into a DataFrame


def parse_records_to_dataframe(root, record_tag: str):
    # tree = ET.parse(xml_file)
    # root = tree.getroot()

    # Define the namespace
    ns = {'ns': 'http://360Scheduling.com/Schema/dsScheduleData.xsd'}

    records = []

    # Extract data from each <record_tag> record
    for record in root.findall(f'ns:{record_tag}', ns):
        record_data = {}
        for element in record:
            tag = element.tag.split('}')[-1]  # Remove namespace if present
            record_data[tag] = element.text
        records.append(record_data)

    # Create DataFrame
    df = pd.DataFrame(records)

    # Convert columns containing 'date' to datetime
    for column in df.columns:
        #if ('date' in column or 'start_time' in column or 'end_time' or 'activity_start' or 'activity_end' in column):
        if ('date' in column or 'start_time' in column or 'end_time' in column or 'activity_start' in column or 'activity_end' in column):
            #column name is different from date_time_fixed
            if column != 'date_time_fixed':
                df[column] = pd.to_datetime(df[column])

    return df

# Function to read a configuration file for Parameters and password

In [3]:
# Function that reads psourl, accountid, user and password parameters  from file config.txt and store in variables
def read_config_file(file: str):
    with open(file) as f:
        lines = f.readlines()
        psourl = lines[0].split('=')[1].strip()
        accountid = lines[1].split('=')[1].strip()
        user = lines[2].split('=')[1].strip()
        password = lines[3].split('=')[1].strip()
    return psourl, accountid, user, password

# Get PSO Authorization Token

In [4]:
# create a function that call a rest api using post and extract the "SessionToken" from the response
def get_session_token(psourl: str, accountid: str, user: str, password: str):

    # Get the PSO token
    url = f"{psourl}/session?accountId={accountid}&userName={user}&password={password}"
    response = requests.post(url)
    response.raise_for_status()
    session_token = response.json()["SessionToken"]
    return session_token

# Get PSO Plan from Rest API

In [6]:
# create a function that call a rest api with get using datasetId as input parameter of the functiona and url and store the respone in planXML variable
def get_plan_xml(datasetId: str, token: str):

    # Get the PSO plan
    url = f"{psourl}/data?datasetId={datasetId}&includeInput=true&includeOutput=true"
    headers = {"Authorization": f"Bearer {
        token}", "Accept": f"application/xml"}
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    planXML = response.text
    return planXML

## Store the token in a global variable

In [5]:
#global token
#psourl, accountid, user, password = read_config_file("../config/config.txt")
#token = get_session_token(psourl, accountid, user, password)

## Test the retrieval of a plan from the server

In [7]:
# test get_plan_xml function
#planXML = get_plan_xml("LXDEMO",token)